# Line Integrated Diagnostics: Interferometry

[LineIntegratedDiagnostic]: ../../api/plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic.rst#plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic

Many plasma diagnostics use a beam of light or particles to probe the properties of a plasma. Such measurements are inherently line-integrated: for example, an imaging interferometer reduces the density along the probe beam path, $n(x,y,z)$, to a 2D plane of phase shifts, $\Delta \phi(x,y)$. 

The [LineIntegratedDiagnostic] class in PlasmaPy provides an abstract framework for creating synthetic diagnostics that work in this manner. These synthetic diagnostics can then be used to predict the results of experimental measurements given an analytical model or simulation result describing a plasma.

Although PlasmaPy contains LineIntegrateDiagnostic subclasses for several diagnostics (including interferometry), this notebook will explicitly create an Interferometer synthetic diagnostic to demonstrate the use of the [LineIntegratedDiagnostic] abstract class.

In [ ]:
import astropy.constants as const
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from plasmapy.diagnostics.path_integrated_diagnostic import LineIntegratedDiagnostic
from plasmapy.plasma.grids import CartesianGrid

## Contents

1. [The Line Integrated Diagnostic Framework](#The-Line-Integrated-Diagnostic-Framework)
1. [Creating the Interferometer Class](#Creating-the-Interferometer-Class)
1. [Using the Interferometer Class](#Using-the-Interferometer-Class)   

## The Line Integrated Diagnostic Framework

[LineIntegratedDiagnostic]: ../../api/plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic.rst#plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic

[CartesianGrid]: ../../api/plasmapy.plasma.grids.CartesianGrid.rst#plasmapy.plasma.grids.CartesianGrid

<img src="line_integrated_diagnostic_setup.png">

A [LineIntegratedDiagnostic] class object is instantiated by providing a [CartesianGrid] object (representing the plasma to be measured) as well as vectors from the origin of the [CartesianGrid] to the detector plane and the source plane. The line integral through the grid is then calculated using the `line_integral` method. This method takes keywords that define the size and resolution of the detector. The line integral is then calculated using the following method:

1. A 2D array of points (Nx, Ny) in the detector plane is created. A corresponding array of points is created in the source plane. If the `collimated` keyword is False, all of the source points will instead be set equal to the point defined by the source vector. 
2. Equations are calculated for lines that connect each point in the detector plane to the corresponding point in the source plane. 
3. A number of points (set by the `num` keyword) are generated on each line in the region where the lines cross the grid. The result is a 3D array with dimensions (Nx, Ny, num).
4. The `integrand()` method is evaluated at each point in the 3D array.
5. The resulting integrand array is then integrated along the last dimension to produce the final (Nx,Ny) array representing the line-integral as collected in the detector plane.

Subclasses of [LineIntegratedDiagnostic] overwrite the `integrand()` method in order to line-integrate different quantities and thus represent different diagnostics.

## Creating the Interferometer Class

[LineIntegratedDiagnostic]: ../../api/plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic.rst#plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic

An interferometer diagnoses plasma density by measuring the phase shift ($\Delta \phi$) of a probe beam. The phase shift is 

\begin{equation}
\Delta \phi = -\frac{\omega_{probe}}{2 c n_c} \int n_e dl
\end{equation}

Where $\omega_{probe}$ is the probe beam frequency, $c$ is the speed of light, $\int n_e dl$ is the line-integrated electron density, and $n_c$ is the critical density

\begin{equation}
n_c = \frac{\epsilon_0 m_e}{e^2} \omega_{probe}^2
\end{equation}

In order to predict the phase shift through a plasma, we therefore only need to know the density line-integrated along the probe beam axis. This can be accomplished by defining a [LineIntegratedDiagnostic] with an appropriate `_integrand()` method as follows.

In [ ]:
class Interferometer(LineIntegratedDiagnostic):
    def _integrand(self, pts):
        # Reshape the pts array from grid shape (nx, ny, nz, 3) to a list
        # of points (nx*ny*nz, 3) as required by the grids interpolators
        nx, ny, nz, ndim = pts.shape
        pts = np.reshape(pts, (nx * ny * nz, ndim))
        
        integrand =  self.grid.volume_averaged_interpolator(pts, "n_e")
        
        # Reshape the integrands from (nx*ny*nz) to (nx, ny, nz)
        integrand = np.reshape(integrand, (nx, ny, nz))

        return integrand

    def evaluate(
        self,
        probe_freq: u.Hz,
        size=np.array([[-1, 1], [-1, 1]]) * u.cm,
        bins=[50, 50],
        collimated=True,
        num=100,
        unwrapped=False,
    ):

        n_c = (
            (const.eps0.si * const.m_e / const.e.si ** 2)
            * (2 * np.pi) ** 2
            * probe_freq ** 2
        ).to(u.cm ** -3)

        hax, vax, int_ne = self._line_integral(
            size=size, bins=bins, collimated=collimated, num=num
        )

        phase_shift = (-np.pi * probe_freq / (const.c.si * n_c)) * int_ne
        phase_shift = phase_shift.to(u.dimensionless_unscaled).value

        if not unwrapped:
            phase_shift = (phase_shift + np.pi) % (2 * np.pi) - np.pi

        return hax, vax, phase_shift

[LineIntegratedDiagnostic]: ../../api/plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic.rst#plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic

The `_integrand()` method will return the line-integrated density. The `evaluate()` method is a wrapper around the `_line_integral()` method of [LineIntegratedDiagnostic] that calculates the total phase shift from the line-integrated density. If the `unwrapped` keyword is not set, then the function takes the modulus with respect to $2 \pi$ to estimate where jumps in the phase will occur.

The resulting `Interferometer` class is equivalent to the implementation in PlasmaPy, although the latter actually makes use of the `LineIntegratedScalarQuantity` subclass of [LineIntegratedDiagnostic].

## Using the Interferometer Class

In order to demonstrate the use of the Interferometer class, we'll create a sphere of constant electron density.

In [ ]:
axis = np.linspace(-2, 2, num=200) * u.mm
xarr, yarr, zarr = np.meshgrid(axis, axis, axis, indexing="ij")
r = np.sqrt(xarr ** 2 + yarr ** 2 + zarr ** 2)
n_e = np.where(r < 1 * u.mm, 1, 0) * 3e19 / u.cm ** 3
grid = CartesianGrid(xarr, yarr, zarr)
grid.add_quantities(n_e=n_e)


fig, ax = plt.subplots(figsize=(6, 6))
ax.pcolormesh(
    axis.to(u.mm).value, axis.to(u.mm).value, grid["n_e"][:, 100, :], shading="auto"
)
ax.set_xlabel("X (mm)")
ax.set_ylabel("Z (mm)")
ax.set_aspect("equal")

Now we can initialize the `Interferometer` class by also defining the locations of the source and detector

In [ ]:
source = (0 * u.mm, -5 * u.mm, 0 * u.mm)
detector = (0 * u.mm, 5 * u.mm, 0 * u.mm)
obj = Interferometer(grid, source, detector, verbose=False)

Finally we will specify the size and resolution of the detector plane when calling the `evaluate()` method to create the synthetic interferogram.

In [ ]:
size = np.array([[-1, 1], [-1, 1]]) * 1.2 * u.mm
bins = [350, 350]

hax, vax, phase = obj.evaluate(1.14e15 * u.Hz, size=size, bins=bins, num=100)
hax = hax.to(u.mm).value
vax = vax.to(u.mm).value

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot = ax.pcolormesh(hax, vax, phase.T / np.pi, cmap="binary", shading="auto")
ax.set_aspect("equal")
ax.set_xlabel("X (mm)", fontsize=16)
ax.set_ylabel("Z (mm)", fontsize=16)

cb = fig.colorbar(plot, ax=ax, orientation="vertical", pad=0.03, shrink=0.7)
cb.ax.set_ylabel("Phase Shift / $\pi$", fontsize=16)

Running the same calculation with `unwrapped=True` shows the total phase shift experienced. As expected, the phase shift is highest at the center of the sphere where the line-integrated density is highest. 

In [ ]:
hax, vax, phase = obj.evaluate(
    1.14e15 * u.Hz, size=size, bins=bins, num=100, unwrapped=True
)
hax = hax.to(u.mm).value
vax = vax.to(u.mm).value

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot = ax.pcolormesh(hax, vax, phase.T, cmap="binary", shading="auto")
ax.set_aspect("equal")
ax.set_xlabel("X (mm)")
ax.set_ylabel("Z (mm)")

cb = fig.colorbar(plot, ax=ax, orientation="vertical", pad=0.03, shrink=0.7)
cb.ax.set_ylabel("Phase Shift (unwrapped)", fontsize=16)